In [8]:
from utils import download_images

In [9]:
import pandas as pd
df = pd.read_csv('../train.csv')

In [10]:
df_10 = df.sample(10, random_state=42)
df_10

,image_link,group_id,entity_name,entity_value
101138,https://m.media-amazon.com/images/I/81nEfmLyyM...,630390,item_weight,1.0 kilogram
165563,https://m.media-amazon.com/images/I/51gz4v5BxT...,751532,width,78.0 inch
228235,https://m.media-amazon.com/images/I/61D2A-LxNn...,478233,height,10.1 inch
197996,https://m.media-amazon.com/images/I/51INxCAF0T...,943839,depth,2.88 inch
107574,https://m.media-amazon.com/images/I/81-zmM+W8B...,487566,item_weight,1.8 pound
243052,https://m.media-amazon.com/images/I/51nHF2ebjc...,970563,height,71.0 centimetre
202803,https://m.media-amazon.com/images/I/61+fi9MKzR...,192007,depth,2.55 inch
59020,https://m.media-amazon.com/images/I/51f7mBQtF7...,507988,item_weight,93.0 pound
215157,https://m.media-amazon.com/images/I/51pbHMuYfS...,816782,depth,120.0 centimetre
225702,https://m.media-amazon.com/images/I/618YCp5wl0...,810266,height,15.0 centimetre


In [11]:
images_links = df_10['image_link'].to_list()

In [7]:
download_images(images_links, "./sample_images", True)

Exception in thread Thread-7 (_handle_workers):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\manas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\multiprocessing\pool.py", line 522, in _handle_workers
    cls._wait_for_updates(current_sentinels, change_notifier)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\L

In [1]:
import cv2
print(cv2.__version__)  # This should print the version of OpenCV installed


4.10.0


In [18]:
from PIL import Image

image_path = 'image_1.png'  # Replace with your image file
try:
    img = Image.open(image_path)
    img.show()  # Display the image if it's valid
    print("Image loaded successfully with Pillow.")
except Exception as e:
    print(f"Error loading image with Pillow: {e}")


Error loading image with Pillow: [Errno 2] No such file or directory: 'C:\\Users\\manas\\OneDrive\\Desktop\\student_resource 3\\src\\image_1.png'


In [17]:
import cv2
import os
import pytesseract
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


# Function to display multiple images in a grid
def display_images(images, title):
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))  # 2 rows, 5 columns
    fig.suptitle(title, fontsize=16)
    
    for i, img in enumerate(images):
        ax = axes[i // 5, i % 5]
        ax.imshow(img, cmap='gray')
        ax.axis('off')
        ax.set_title(f'Image {i+1}')
    
    plt.tight_layout()
    plt.show()

# Initialize an array to store the images
images = []

# Define the number of images (10 in this case)
num_images = 10

# Path where the images are stored
image_paths = [img for img in os.listdir(r'C:\\Users\\manas\\OneDrive\\Desktop\\student_resource 3\\src\\sample_images')]  # Assuming image_1.png, image_2.png, ...
original_images = []
# 1. Load and display original images
for path in image_paths:
    img = Image.open(path)
    if img is not None:
        original_images.append(img)  # Image is loaded correctly
    else:
        original_images.append(None) 
# original_images = images_arr
display_images(original_images, "Original Images")

# 2. Convert to grayscale and display
gray_images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in original_images]
display_images(gray_images, "Grayscale Images")

# 3. Apply Otsu's Binarization and display
binary_images = [cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] for gray in gray_images]
display_images(binary_images, "Binarized Images (Otsu's)")

# 4. Noise removal using Median Blur and display
denoised_images = [cv2.medianBlur(binary, 3) for binary in binary_images]
display_images(denoised_images, "Denoised Images (Median Blur)")

# 5. Dilation and display
kernel = np.ones((1, 1), np.uint8)
dilated_images = [cv2.dilate(denoised, kernel, iterations=1) for denoised in denoised_images]
display_images(dilated_images, "Dilated Images")

# 6. Erosion and display
eroded_images = [cv2.erode(dilated, kernel, iterations=1) for dilated in dilated_images]
display_images(eroded_images, "Eroded Images")

# 7. Skew Correction (rotation) and display
def correct_skew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

skew_corrected_images = [correct_skew(eroded) for eroded in eroded_images]
display_images(skew_corrected_images, "Skew Corrected Images")

# 8. Resize images (increase size for better OCR performance)
resized_images = [cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC) for image in skew_corrected_images]
display_images(resized_images, "Resized Images")

# 9. Invert colors for white text on black background (optional, based on image content)
inverted_images = [cv2.bitwise_not(image) for image in resized_images]
display_images(inverted_images, "Inverted Images (Optional)")

# 10. OCR Processing on each preprocessed image
ocr_results = []
for img in inverted_images:
    text = pytesseract.image_to_string(img)
    ocr_results.append(text)

# Output OCR results for each image
for idx, text in enumerate(ocr_results):
    print(f"Text from image {idx+1}:\n{text}\n")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\manas\\OneDrive\\Desktop\\student_resource 3\\src\\41hriA7iZ+L.jpg'

In [20]:
import os

file_path = r'C:\Users\manas\OneDrive\Desktop\student_resource 3\src'

try:
    # Check if the file exists before opening it
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            print("File opened successfully.")
    else:
        print("File not found.")
except FileNotFoundError as e:
    print(f"Error: {e}")


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\manas\\OneDrive\\Desktop\\student_resource 3\\src'